In [2]:
#importing
import torch
import torchvision
from optuna.terminator.improvement.emmr import torch
from torch.utils.data import DataLoader , Subset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split



C:\Users\HP\PycharmProjects\PythonProject\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Random_seed = 123
learning_rate = 0.0001
Epoch = 10
batch_size = 32
num_classes = 10

In [5]:
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
import random
import os
import torch

class ArtBenchTriplet(Dataset):
    def __init__(self, csv_path, img_dir, split='train', transform=None):
        # 1. Load the metadata CSV
        self.df = pd.read_csv(csv_path)

        # 2. Filter by 'train' or 'test' using the column you mentioned
        self.df = self.df[self.df['split'] == split].reset_index(drop=True)

        self.img_dir = img_dir
        self.transform = transform

        # 3. Create a map of {label: [indices]} so we can find positives quickly
        # This makes sure 'Positive' is always the same style as 'Anchor'
        self.label_to_indices = self.df.groupby('label').groups

    def __getitem__(self, index):
        # --- GET ANCHOR ---
        row = self.df.iloc[index]
        label = row['label']
        anchor_path = os.path.join(self.img_dir, row['path'])
        anchor_img = Image.open(anchor_path).convert('RGB')

        # --- GET POSITIVE (Same style, different image) ---
        # We pick a random index from the same label group
        pos_indices = self.label_to_indices[label]
        pos_index = random.choice(pos_indices)
        # Ensure positive isn't the exact same image as anchor
        while pos_index == index:
            pos_index = random.choice(pos_indices)

        pos_row = self.df.iloc[pos_index]
        pos_path = os.path.join(self.img_dir, pos_row['path'])
        pos_img = Image.open(pos_path).convert('RGB')

        # --- GET NEGATIVE (Different style) ---
        # Pick a style that is NOT the current label
        all_labels = list(self.label_to_indices.keys())
        neg_label = random.choice([l for l in all_labels if l != label])

        neg_index = random.choice(self.label_to_indices[neg_label])
        neg_row = self.df.iloc[neg_index]
        neg_path = os.path.join(self.img_dir, neg_row['path'])
        neg_img = Image.open(neg_path).convert('RGB')

        # Apply transforms (Resize, Normalize, etc.)
        if self.transform:
            anchor_img = self.transform(anchor_img)
            pos_img = self.transform(pos_img)
            neg_img = self.transform(neg_img)

        return anchor_img, pos_img, neg_img

    def __len__(self):
        return len(self.df)

In [6]:
#Data Loading
train_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((125,125)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
])
Val_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((125,125)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
])



print(f"Train loader has {len(train_loader)} batches.")
print(f"Valid loader has {len(valid_loader)} batches.")
for images, labels in train_loader:
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    print('Class labels of 10 examples:', labels[:10])
    break


NameError: name 'train_loader' is not defined

In [13]:
#Residual Block

class ResidualBlock(torch.nn.Module):
    def __init__(self,channels):
        super(ResidualBlock, self).__init__()
        self.block =  torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=channels[0], out_channels=channels[1], kernel_size=(3,3), stride=(2,2), padding=1),
            torch.nn.BatchNorm2d(channels[1]),
            torch.nn.ReLU(inplace=True),
              torch.nn.Conv2d(in_channels=channels[1],
                                out_channels=channels[2],
                                kernel_size=(1, 1),
                                stride=(1, 1),
                                padding=0),
                torch.nn.BatchNorm2d(channels[2])
        )

        self.shortcut = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels=channels[0],
                                out_channels=channels[2],
                                kernel_size=(1, 1),
                                stride=(2, 2),
                                padding=0),
                torch.nn.BatchNorm2d(channels[2])
        )

    def forward(self,x):
        shortcut = x
        block = self.block(x)
        shortcut = self.shortcut(x)
        x = torch.nn.functional.relu(block + shortcut)
        return x


In [14]:
# Model

class ConvNet(torch.nn.Module):
    def __init__(self,num_classes):
        super(ConvNet, self).__init__()
        self.residual_block1 = ResidualBlock(channels = [3,32,64])
        self.residual_block2 = ResidualBlock(channels = [64,64,128])
        self.residual_block3 = ResidualBlock(channels = [128,128,256])
        self.residual_block4 = ResidualBlock(channels = [256,256,512])
        self.gap = torch.nn.AdaptiveAvgPool2d((1, 1))

        self.linear_1 = torch.nn.Linear(512,num_classes)

    def forward(self,x):
        x = self.residual_block1(x)
        x = self.residual_block2(x)
        x = self.residual_block3(x)
        x = self.residual_block4(x)
        x = self.gap(x)
        x = torch.flatten(x,1)
        logits = self.linear_1(x)
        return logits

torch.manual_seed(Random_seed)
model = ConvNet(num_classes=num_classes)

model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr= learning_rate,weight_decay=0.02)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,mode = 'min', factor =  0.1 ,patience= 5)

In [15]:
def compute_accuracy(model, data_loader):
    model.eval()
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
        features = features.to(device)
        targets = targets.to(device)

        logits = model(features)
        # Get the index of the highest logit (this is the predicted class)
        _, predicted_labels = torch.max(logits, 1)

        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()

    return correct_pred.float() / num_examples * 100

In [16]:
#Training
for epoch in range(Epoch):
    model = model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):

        features = features.to(device)
        targets = targets.to(device)

        ### FORWARD AND BACK PROP
        logits = model(features)
        cost = torch.nn.functional.cross_entropy(logits, targets)
        optimizer.zero_grad()

        cost.backward()

        ### UPDATE MODEL PARAMETERS
        optimizer.step()
    scheduler.step()

        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f'
                   %(epoch+1, Epoch, batch_idx,
                     len(train_loader)//batch_size, cost))

    model = model.eval() # eval mode to prevent upd. batchnorm params during inference
    with torch.set_grad_enabled(False): # save memory during inference
        print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
              epoch+1, Epoch,
              compute_accuracy(model, train_loader)))


C:\Users\HP\AppData\Local\Temp\ipykernel_29708\633328319.py:21: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\autograd\generated\python_variable_methods.cpp:837.)
  print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f'


Epoch: 001/010 | Batch 000/020 | Cost: 2.3638
Epoch: 001/010 | Batch 050/020 | Cost: 2.0368
Epoch: 001/010 | Batch 100/020 | Cost: 2.0118
Epoch: 001/010 | Batch 150/020 | Cost: 1.7506
Epoch: 001/010 | Batch 200/020 | Cost: 1.8657
Epoch: 001/010 | Batch 250/020 | Cost: 2.0151
Epoch: 001/010 | Batch 300/020 | Cost: 1.9258
Epoch: 001/010 | Batch 350/020 | Cost: 1.9049
Epoch: 001/010 | Batch 400/020 | Cost: 1.5892
Epoch: 001/010 | Batch 450/020 | Cost: 1.5795
Epoch: 001/010 | Batch 500/020 | Cost: 1.5413
Epoch: 001/010 | Batch 550/020 | Cost: 1.8173
Epoch: 001/010 | Batch 600/020 | Cost: 1.5411
Epoch: 001/010 | Batch 650/020 | Cost: 1.6498
Epoch: 001/010 training accuracy: 42.76%
Epoch: 002/010 | Batch 000/020 | Cost: 1.6775
Epoch: 002/010 | Batch 050/020 | Cost: 1.6280
Epoch: 002/010 | Batch 100/020 | Cost: 1.4862
Epoch: 002/010 | Batch 150/020 | Cost: 1.7907
Epoch: 002/010 | Batch 200/020 | Cost: 1.4858
Epoch: 002/010 | Batch 250/020 | Cost: 1.6363
Epoch: 002/010 | Batch 300/020 | Cost: 